# Notebook to play with SMAs

Latest version: 2024-08-16  
Author: MvS

## Description


## Result




In [ ]:
import yfinance
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

periods = [200, 50]

dt_end = datetime.datetime.today()
dt_data_start = dt_end - datetime.timedelta(days=max(periods) * 3)

stock = "FROG"

# Grab the stock data
load_df = yfinance.download(
    stock, start=dt_data_start.strftime('%Y-%m-%d'), end=dt_end.strftime('%Y-%m-%d')
)

In [ ]:
stock_df = load_df.copy()

# Compute the simple moving average (SMA)
for period in periods:
    stock_df[f"SMA_{period:03}"] = stock_df["Close"].rolling(window=period).mean()

# Now that we calculated the SMA, we can remove the data points before the actual AOI that we want.
stock_df = stock_df[-max(periods) :].copy()


dt_start = stock_df.index[0]

print(
    f"""Calculating SMAs for {periods} length starting from:
  {dt_start.strftime('%Y-%m-%d')} and ending on {dt_end.strftime('%Y-%m-%d')}
  and containing {stock_df.shape} data points."""
)

stock_df[[f"SMA_{min(periods):03}", f"SMA_{max(periods):03}", 'Close']].plot()

In [ ]:
stock_df['C'] = 0
stock_df.drop(['C'], axis=1, inplace=True)
stock_df[-5:]

In [ ]:
print(stock_df['Close'][-50:].mean())
print(stock_df['SMA_050'][-50:].mean())
print(stock_df['SMA_200'][-50:].mean())

In [ ]:
# define the corridors for operation as strong deviations of fast from slow SMA


def valid_signal(x):
    fast = f"SMA_{min(periods):03}"
    slow = f"SMA_{max(periods):03}"

    # mean of last 50 days
    ref_price = stock_df[fast][-50:].mean()
    perc = 0.01 * ref_price

    if x[fast] > x[slow] + perc:
        return 1
    elif x[fast] < x[slow] - perc:
        return -1
    else:
        return 0


# don't need loc unless working with view/slice of df
# stock_df.loc[:, 'Valid'] = stock_df.apply(valid_signal, axis=1)
stock_df['Valid'] = stock_df.apply(valid_signal, axis=1)

stock_df[[f"SMA_{min(periods):03}", f"SMA_{max(periods):03}", 'Valid']].plot()

In [ ]:
stock_df['Valid_shift'] = stock_df['Valid'].shift(1)
stock_df['Valid_shift'] = stock_df['Valid_shift'].interpolate(
    method='backfill', limit_direction='backward'
)


def get_signal(x):
    if x['Valid'] != x['Valid_shift'] and x['Valid'] != 0:
        return x['Valid'] * 5
    else:
        return 0


stock_df['Signal'] = stock_df.apply(get_signal, axis=1)

stock_df[
    [f"SMA_{min(periods):03}", f"SMA_{max(periods):03}", 'Valid', 'Signal', 'Close']
].plot()

In [ ]:
if stock_df['Signal'].any() != 0:
    print('Some signals found...')
    # calculate timedeltas between signals
    signals_df = stock_df[stock_df['Signal'] != 0].copy()
    signals_df['Temp'] = signals_df.index
    signals_df['Last_Signal'] = signals_df.index - signals_df['Temp'].shift(1)

    # calculate first row using info from remaining stock data
    signals_df.iat[0, signals_df.columns.get_loc('Last_Signal')] = signals_df.iat[
        0, signals_df.columns.get_loc('Temp')
    ] - min(stock_df.index)

    signals_df

In [ ]:
# clean up
# stock_df.dtypes
stock_df.drop(
    ['Last_Signal', 'Temp', 'Valid_shift', 'Adj Close'],
    axis=1,
    inplace=True,
    errors='ignore',
)
stock_df = stock_df.astype({'Signal': int})

if stock_df['Signal'].any() != 0:
    # add back the signal calculation
    stock_df = pd.concat([stock_df, signals_df[['Last_Signal']]], axis=1)
    stock_df['Last_Signal'] = stock_df['Last_Signal'].fillna(datetime.timedelta(days=0))
else:
    stock_df['Last_Signal'] = datetime.timedelta(days=0)

stock_df

In [ ]:
stock_df2 = load_df.copy()
periods = [50, 200]
# Compute the simple moving average (SMA)
for period in periods:
    stock_df2[f"SMA_Vol_{period:03}"] = (
        stock_df2["Volume"].rolling(window=period).mean()
    )

# Now that we calculated the SMA, we can remove the data points before the actual AOI that we want.
stock_df2 = stock_df2[-max(periods) :].copy()


dt_start = stock_df2.index[0]

print(
    f"""Calculating SMAs for {periods} length starting from:
  {dt_start.strftime('%Y-%m-%d')} and ending on {dt_end.strftime('%Y-%m-%d')}
  and containing {stock_df2.shape} data points."""
)

stock_df2[[f"SMA_Vol_{min(periods):03}", f"SMA_Vol_{max(periods):03}", 'Volume']].plot()

In [ ]:
# stock_df['LastSignal'] =
stock_df.index.to_series().mask(stock_df['Signal'].eq(0))
# .eq(0)).fill()-stock_df.index

# stock_df[stock_df['Signal'] != 0]

In [ ]:
df = pd.DataFrame(
    {
        'index': [240, 251, 282, 301, 321, 325, 328, 408],
        'a': ['0', '0', '0', '0', '1', '0', '0', '1'],
        'e1': ['0', '1', '0', '0', '0', '0', '1', '0'],
        'e2': ['1', '0', '1', '0', '0', '1', '0', '0'],
    }
)
df.set_index('index', inplace=True)
dfTarget = pd.DataFrame(
    {
        'index': [240, 251, 282, 301, 321, 325, 328, 408],
        'a': ['0', '0', '0', '0', '1', '0', '0', '1'],
        'e1': ['0', '70', '0', '0', '0', '0', '80', '0'],
        'e2': ['81', '0', '39', '0', '0', '83', '0', '0'],
    }
)
dfTarget.set_index('index', inplace=True)

In [ ]:
cols = ['e1', 'e2']
s = df.index.to_series().mask(df['a'].ne('1')).bfill() - df.index

df[cols] = df[cols].mask(df.eq('1'), s, axis=0).astype(int)
df

In [ ]:
fast_signal = stock_data['SMA_050']
slow_signal = stock_data['SMA_200']

# Create a boolean series where slow > fast
fast_condition = fast_signal > slow_signal
slow_condition = fast_signal <= slow_signal

window_size = 2

# Calculate if this condition was met at least once in the last n periods
earlier_fast_condition = fast_condition.rolling(window=window_size, min_periods=1).min()
earlier_slow_condition = slow_condition.rolling(window=window_size, min_periods=1).min()


# Function to include the rolling check
def _conditions(slow, fast, prev_slow, prev_fast):
    signal_strength = 5
    # if fast > slow and not prev_slow:
    #     return signal_strength
    # elif fast <= slow and not prev_fast:
    #     return -signal_strength
    # else:
    #     return 0
    if fast > slow and not prev_fast:
        return signal_strength
    elif fast <= slow and not prev_slow:
        return -signal_strength
    else:
        return 0


# Function to include the rolling check
def _conditions_new(slow, fast, prev_slow, prev_fast):
    signal_strength = 7
    if prev_fast:
        return signal_strength
    else:
        return 0


# Vectorize the function
func = np.vectorize(_conditions)
func_new = np.vectorize(_conditions_new)

# Create a new column based on the function, considering rolling windows
stock_data.loc[:, 'Signal'] = func(
    slow_signal, fast_signal, earlier_slow_condition, earlier_fast_condition
)
# Create a new column based on the function, considering rolling windows
stock_data.loc[:, 'Signal_new'] = func_new(
    slow_signal, fast_signal, earlier_slow_condition, earlier_fast_condition
)

# Alternate syntax
# list_args = [sales, profit]
# df["rank4"] = func(*list_args)
stock_data[['Close', 'SMA_050', 'SMA_200', 'Signal', 'Signal_new']].plot()

In [ ]:
print(slow_signal.rolling(window=window_size, min_periods=1))

In [ ]:
stock_data[['Close', 'SMA_050', 'SMA_200']]
# validated in Trading View